### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [1]:
# poetry add python-dotenv langchain langchain-openai

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_4


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 인터넷 글을 방대하게 수집해 토큰 단위로 자른 뒤, 다음 토큰을 맞히는 ‘예측 게임’을 반복하며 스스로 패턴을 '
 '학습합니다.  \n'
 '학습이 끝나면 주어진 질문을 토큰 시퀀스로 받아, 각 위치에서 가장 적절한 단어의 확률을 계산해 한 토큰씩 순차적으로 생성합니다.  \n'
 '이렇게 얻은 확률 분포를 바탕으로 사람이 이해할 만한 자연스러운 문장을 만들어내는 것이 핵심 원리입니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [17]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('ChatGPT는 인터넷의 방대한 텍스트를 학습해 단어 확률을 익힌 뒤, 사람이 준 프롬프트에 가장 자연스러운 다음 말을 예측·생성하는 '
 '방식으로 작동합니다.  \n'
 '학습 과정에선 인간의 대화 데이터로 미세 조정(강화학습·인간 피드백)을 거쳐 사실성·윤리성을 높입니다.  \n'
 '결과적으로 주어진 문맥을 기억해 논리적이고 유창한 응답을 이어가는 대화형 언어 모델이 됩니다.\n'
 '\n'
 'ChatGPT의 주요 장점  \n'
 '- 다양한 주제에 대한 이해도와 유창한 문장 생성  \n'
 '- 질문·번역·요약·코드 작성 등 다양한 업무를 하나의 모델로 처리  \n'
 '- 대화 맥락을 유지해 자연스러운 연속 대화 가능  \n'
 '- 사용자 피드백을 반영해 지속적으로 개선되며, 즉각적이고 24시간 이용 가능\n'
 '\n'
 'ChatGPT와 비슷한 한국어 AI 모델: 구글 바드(제미나이), 메타 람마, 에이디티 누스(Anthropic 클라우드), 카카오 코랑, '
 '네이버 클로바X, 카카오브레인 코기(1.0/2.0) 등')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "claude", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
# 각 딕셔너리 q를 언패킹해 .format의 field를 채움
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 대규모 텍스트 데이터를 바탕으로 다음 단어를 예측하도록 사전 학습된 트랜스포머 기반 언어 모델입니다.  \n'
 '지도 학습 방식으로 인간이 작성한 프롬프트-응답 쌍을 반복해서 학습해, 사람이 선호하는 답변 패턴을 점점 모방합니다.  \n'
 '강화학습과 인간 피드백(RLHF)을 추가로 거치면서 안전성·유용성·정직성을 높인 최종 모델이 완성됩니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 대규모 텍스트·이미지·오디오 등 다중 모달 데이터를 동시에 처리하도록 설계된 전이 학습 기반의 거대 신경망 모델이다.  \n'
 '먼저 인터넷 긁기, 책 스캔, 유튜브 자막 등 방대한 공개 데이터를 바탕으로 다음 토큰을 예측하는 일반 사전 학습을 수행한다.  \n'
 '이후 인간의 선호에 맞춰 답변 품질을 높이도록 강화학습·인간 피드백(RLHF)을 반복해 미세 조정하며, Chain-of-Thought '
 '프롬프트를 통해 다단계 추론 능력도 강화한다.  \n'
 '결과적으로 Gemini는 텍스트 생성, 코드 작성, 이미지 분석, 수학·과학 문제 풀이 등 다양한 작업을 단일 모델로 처리할 수 있게 '
 '된다.')
<class 'str'> claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 방대한 텍스트를 통해 다음 토큰을 예측하도록 사전학습되고, 인간 대화 샘플로 강화학습·인간 피드백(RLHF)으로 가치를 '
 '맞춘다.  \n'
 "이어서 '헌법(Constitutional AI)'이라는 원칙 집합을 스스로 참고하게 하여 유해·편향 답변

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [ ]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. \
     Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)
# 출력은 구조화된 AIMessage 타입이며, 내용을 쓰려면 response.content로 접근
print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI.      Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “GPT(Generative Pre-trained Transformer)” 계열 모델의 한 버전으로,  
“**미리 학습(pre-train) → 미세 조정(fine-tune)**”이라는 2단계 흐름을 기반으로 만들어집니다.  
핵심 원리는 **“주어진 앞 단어들을 보고 다음 단어를 예측하는(=확률을 계산하는) 방대한 자기 회귀(self-supervised) 언어 모델”**입니다.  
이 과정을 단계별로 풀어 보면 다음과 같습니다.

--------------------------------------------------
1. 자기 지도 학습(Self-Supervised Learning) : Pre-training
--------------------------------------------------
1) 데이터  
   - 공개 웹(웹페이지, 위키, 책, 깃헙 등)에서 수 TB 급의 텍스트  
   - 별도로 “정답”을 만들 필요 없음 → 텍스트 자체가 레이블이 됨

2) 학습 목표  
   - 입력 시퀀스의 다음 토큰(token, 단어·부분 단어·기호)을 맞추는 최대 우도(MLL, Maximum Likelihood)  
   - 수식화 : max Σ log P(wᵢ | w₁…wᵢ₋₁; θ)  
   - 결과적으로 웬만한 지식·문법·상식이 모델 파라미터(θ)에 압축됨

In [7]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT(또는 GPT 계열 모델)가 “왜 질문에 대해 그런 답변을 할 수 있는가?”는 단 한 가지 학습 목표에서 시작됩니다.  
“주어진 앞쪽 토큰(token)들을 보고 다음 토큰을 맞춰라.”  
이 단순한 과제를 대규모 신경망, 대규모 데이터, 대규모 컴퓨팅으로 반복하면 ‘대화 능력’이 자연스럽게 생겨납니다. 아래에 1) 학습 단계별로, 2) 핵심 원리별로 나누어 설명하겠습니다.

----------------------------------------
1. 학습 단계별 흐름
----------------------------------------
1. Pre-training (사전 학습)  
   - 데이터 : 공공 웹, 위키피디아, 책, GitHub 등 수천억 토큰  
   - 목표 : 다음 토큰 예측 (자기 회귀 언어 모델링)  
   - 결과물 : 기본(Base) LM → ‘세계 지식’은 있지만 ‘사용자의 뜻’을 잘 따르지 못함.

2. SFT (Supervised Fine-Tuning, 지도 미세 조정)  
   - 데이터 : 사람이 작성한 (프롬프트, 이상적인 답변) 쌍 1~10만 개  
   - 목표 : “좋은 답변이라고 판단되는 텍스트”를 모방하도록 추가 학습  
   - 결과물 : 지시-응답(Instruction-Following) 모델.

3. Reward Modeling (보상 모델 학습)  
   - 데이터 : 같은 프롬프트에 대한 여러 답변을 사람이 1~5 점 또는 순위 매김  
   - 목표 : 사람의 선호를 예측하는 RM(보상 모델)을 만듦.

4. RLHF (Reinforcement Learning from Human Feedback)  
   - 알고리즘 : PPO (Proximal Policy Optimization)  
   - 목표 : RM이 주는 보상 점수를 높이도록 SFT 모델의 파라미터를 미세 조정.  
   - 결과물 : 사람이 더 선호하는 답변을 생성하는 정책(policy) 획득.



#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [8]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [9]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [10]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성(수성→목성까지)을 “거리-크기-특징” 한 줄로 압축하면 다음과 같습니다.

1. **수성** : 태양에서 가장 가까고 작으며, 표면이 달처럼 운석坑 가득하다.  
2. **금성** : 지구와 유사한 크기이나 온실효과로 470 °C 돌아 구리는 ‘지옥행성’이다.  
3. **지구** : 유일한 액체 물이 흐르고 생명이 존재하는 행성이다.  
4. **화성** : 붉은 철鏽 덮인 사막행성으로, 얼음이 있고 인류 탐사의 다음 목표다.  
5. **목성** : 태양계 최대 기체행성으로, 대적점·목성고리·79개 위성(가니메데 등)을 품었다.  
6. **토성** : 아름다운 고리와 82개 위성(타이탄·농밀 대기)을 지닌 기체거인이다.  
7. **천왕성** : 옆으로 누워 자전하며, 淡青色의 얼음행성으로 고리와 27개 위성이 있다.  
8. **해왕성** : 태양계 최강 바람(초속 600 m)이 부는 푸른 얼음행성으로 14개 위성(트리톤 등)을 거느렸다.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [11]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(llm)
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001B13BB5A180> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B13C0935C0> root_client=<openai.OpenAI object at 0x000001B139080BC0> root_async_client=<openai.AsyncOpenAI object at 0x000001B13BBCC740> model_name='moonshotai/kimi-k2-instruct-0905' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**:

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [12]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? \
        {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요?         가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 주로 발생하는 **지구과학 현상** 3가지를 정확히 말씀드리면:

1. **대기냉각(大氣冷卻)**: 여름 동안 가열된 육지가 빠르게 식으면서 기압 차가 커지고, **가을철 큰 일교차**와 **고기압의 발달**이 뚜렷해집니다.  
2. **가을철 해일(秋の高潮)**: 태풍이나 저기압의 영향으로 **가을철 해일**이 자주 발생합니다. 특히 9~10월에 태풍이 북상하면서 **스톰서지(storm surge)**가 유발됩니다.  
3. **황사(黃砂)의 재분출**: 중국 내몽골 지역에서 **가을철 건조한 기류**로 인해 먼지가 일어나 **황사**가 발생하기도 합니다. 봄철보다는 빈도가 낮지만, **가을 황사**도 관측됩니다.

---

**태풍 발생**은 **여름~가을(7~9월)**에 걸쳐서 활발하지만, **가을에만 특정되는 현상은 아닙니다.**  
따라서 **“가을에 일어나는 대표적인 지구과학 현상 = 태풍 발생”**이라고 말하기보다는,  
**“가을철 해일”**이나 **“대기냉각”** 같은 계절적 특성이 더 정확합니다.


In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season()  # 계절 값 얻기
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [ ]:
# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatOpenAI(
#     #api_key=OPENAI_API_KEY,
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.0
# )

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")


 가을에 발생하는 자연 현상:
가을철에 뚜렷하게 나타나는 대표적인 지구과학 현상 3가지는 다음과 같습니다.

1. 시베리아 고기압 확장  
   아시아 대륙이 급속히 식으면서 시베리아 상공에 강한 고기압이 형성되고, 이 고기압이 한반도까지 남하하면서 맑고 건조한 가을 날씨를 만듭니다. 이 현상이 주로 9~10월에 뚜렷해집니다.

2. 노을·단풍 현상  
   고위도에서 내려온 건조한 공기가 맑은 날씨를 유지하면서 해질녘 대기 중 먼지·연기 등 에어로졸이 증가해 붉은 노을이 잘 발생합니다. 동시에 기온이 낮아지면서 단풍나무 등에서 안토시아닌 색소가 합성되어 단풍이 절정을 이룹니다.

3. 계절풍(동북계절풍) 정착  
   대륙과 해양 사이의 온도 차가 커지면서 북서~북동에서 불어오는 차고 건조한 계절풍이 안정적으로 한반도에 정착합니다. 이 바람은 가을철 낮은 습도와 강한 일교차를 만들어 주요 기후 특징으로 작용합니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [15]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1401.17원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: 환율: 1달러 = 1,401.17원  
(기준일: 2024-06-XX, 뱅크오브코리아 기준 종가 기준)

1. 수치적 의미
- ‘1400원 돌파’는 2022년 11월 이후 1년 7개월 만의 사상 두 번째 고점(최고치 1,452.10원/2022-10-11).
- 연초(1,260원 대) 대비 약 11% 절상, 3개월 전(4월 초 1,320원) 대비에도 6% 이상 급등.
- 2022년 고점 대비 아직 3.5% 남은 ‘여유’가 있지만, 1,400원은 심리적 저항선이자 수입·수출 기업의 손익 분기점이 되는 키 레벼.

2. 급등의 3가지 핵심 동인
(1) 달러 강세(DXY 105→107↑)  
- 미 연준의 ‘higher-for-longer’ 어조 재확인(6월 FOMC 닷플롯 3.75→4.0% 상향)  
- 유럽 ECB 선제 인하 기대 vs. FRB 동결 시 차익거래(캐리) 유입  
→ 달러 인덱스 상승 → 원화 약세

(2) 한국 내 외화 유동성 긴장  
- 5월 이후 수출 호조(반도체 50%↑, 자동차 9%↑)에도 불구,  
  · 중동·러시아 프로젝트 건설 자재 선적로 인한 실수요 증가  
  · 해외 증시 사상 최고가에 따른 개인 해외주식 순매수(1~5월 누적 120억달러)  
→ 은행 외화대출·한국증권예탁원 예택잔 고갈 → 스왑포인트 급등(1년 300bp→420bp)

(3) 중국 경기 둔화 & 위안화 약세(USD/CNY 7.10→7.25)  
- 한국 수출의 25%가 중국행 → 위안화 약세는 수콘(수출 경쟁력) 저하 우려 → 외국인 자금 이탈 → 원화 약세 악순환

3. 거시·금융시장 영향
- 물가: 수입물가지수(달러표시) 1% 올라도 CPI 0.04%p 상승. 1400원 지속 시 연간 CPI +0.3%p 우려.
- 기업 실적: 1,350원→1,400원(3.7%↑), 수입원가 1% 상승 → 제조업 영업이익 –0.8%p 하락.
- 증